In [1]:
import glob
import re

import pandas as pd
from utils.utils import get_project_root
import numpy as np

In [2]:
# get all file names
p = get_project_root()
path = get_project_root() / "data/raw/json"
files = glob.glob(f"{path}/**.json")
path_to_data = get_project_root() / "data"

In [3]:
# create a dataframe from it
dfs = []  # an empty list to store the data frames
errs = []
for file in files:
    try:
        data = pd.read_json(file, lines=True, encoding="iso-8859-1")  # read data frame from json file
        dfs.append(data)  # append the data frame to the list
    except Exception as e:
        # these are incomplete files -> parsing failed at some point during save
        errs.append((file, e))

df_original = pd.concat(dfs, ignore_index=True)

In [9]:
# drop local_path
df = df_original.drop("local_path", axis=1)
# drop english articles
df = df[df['language'] == "de"]
# drop duplicates
df.drop_duplicates(subset=["filename", "title"])
# turn date columns into datetime format
df[df.loc[:, df.columns.str.contains("date")].columns] = df.loc[:, df.columns.str.contains("date")].apply(
    pd.to_datetime)
# turn language and source_domain into categoricals
df[["language", "source_domain"]] = df[["language", "source_domain"]].astype("category")
# convert text-columns into text (from object) and remove white spaces
df[["description", "filename", "title", "title_page", "title_rss", "text_main"]].astype("string").apply(
    lambda x: x.str.strip())
df.head()
# TODO unternehmen suchen
# apple? microsoft?

# spiegel paywall blocked many concurrent calls -> remove body texts on these
df.loc[df['text_main'].str.contains("Freier Zugang zu allen Artikeln", na=False), ["text_main"]] = np.NaN
df.loc[df['text_main'].str.contains("Zugang wird gerade auf einem anderen Gerät genutzt", na=False), [
    "text_main"]] = np.NaN

# clean \x und andere ungültige zeichen
df = df.replace('\x84', '"', regex=True)
df = df.replace('\x93', '"', regex=True)

# split authors into individual columns
df = df.join(df["authors"].apply(lambda authors: pd.Series(
    [author.strip() for auth_string in authors for author in re.split("und", auth_string, flags=re.I)])).add_prefix(
    "author_"))
# split categories
df = df.join(df["filename"].apply(lambda row: pd.Series(row.split("_")[:-1])).add_prefix("category_"))
# group and re-sort columns
column_names = [x for x in df.columns]
title_columns = [x for x in df.columns if "title" in x]
text_columns = ["description", "text_main"]
misc_columns = ['language', "source_domain", 'filename', 'url', 'image_url']
author_columns = [x for x in df.columns if "author" in x]
category_columns = [x for x in df.columns if "category" in x]
date_columns = [x for x in df.columns if "date" in x]
df = df[title_columns + text_columns + misc_columns + category_columns + date_columns + author_columns]
df


C:\Users\LeonWolf\AppData\Local\Temp/ipykernel_3944/2173991852.py:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = df.join(df["authors"].apply(lambda authors: pd.Series(


,title,title_page,title_rss,description,text_main,language,source_domain,filename,url,image_url,...,author_3,author_4,author_5,author_6,author_7,author_8,author_9,author_10,author_11,author_12
0,Schriftstellerin Joan Didion mit 87 Jahren ges...,Schriftstellerin Joan Didion mit 87 Jahren ges...,Schriftstellerin Joan Didion mit 87 Jahren ges...,Die Essayistin und Schriftstellerin Joan Didio...,Die Essayistin und Schriftstellerin Joan Didio...,de,www.faz.net,aktuell_feuilleton_buecher_autoren_schriftstel...,https://www.faz.net/aktuell/feuilleton/buecher...,https://media1.faz.net/ppmedia/aktuell/8915247...,...,Joachim Müller-Jung,Maja Brankovic,Dyrk Scherff,Friedrich Schmidt,Gerald Felber,Kim Maurus,NaN,NaN,NaN,NaN
1,"J.R. Moores' Buch ""Electric Wizards""","J.R. Moores' Buch ""Electric Wizards""","J.R. Moores' Buch ""Electric Wizards""",Heute schon Pigs Pigs Pigs Pigs Pigs Pigs Pigs...,Heute schon Pigs Pigs Pigs Pigs Pigs Pigs Pigs...,de,www.faz.net,aktuell_feuilleton_buecher_rezensionen_sachbuc...,https://www.faz.net/aktuell/feuilleton/buecher...,https://media1.faz.net/ppmedia/aktuell/1167631...,...,Ernst-Ludwig Winnacker,Peter-Philipp Schmitt,Christiane Pöhlmann,Georg Imdahl,NaN,NaN,NaN,NaN,NaN,NaN
2,Dating-Seite für Impfgegner: Die ungeimpfte Liebe,Dating-Seite für Impfgegner: Die ungeimpfte Liebe,Unterwegs auf Impffrei:Love: Die ungeimpfte Liebe,"Auf einer neuen Dating-Seite suchen Einsame, d...","Die Menschen wollen lieben. Selbst die, die ha...",de,www.faz.net,aktuell_feuilleton_dating-seite-fuer-impfgegne...,https://www.faz.net/aktuell/feuilleton/dating-...,https://media1.faz.net/ppmedia/aktuell/1238540...,...,Jan Hauser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Frank Ulrich Montgomery verteidigt seine Richt...,Frank Ulrich Montgomery verteidigt seine Richt...,Montgomerys Mätzchen: Ihr seid ja nur eifersüc...,Drehungen eines Unbelehrbaren: Nach Kritik an ...,Ungern möchte man stören bei dem langen Lauf z...,de,www.faz.net,aktuell_feuilleton_debatten_frank-ulrich-montg...,https://www.faz.net/aktuell/feuilleton/debatte...,https://media1.faz.net/ppmedia/aktuell/politik...,...,Helene Bubrowski,Gregor Brunner,Grafiken,Oliver Schlömer,Eckart Lohse,Markus Wehner,Michaela Wiegel,Thomas Geiger,NaN,NaN
4,Impfpflicht jetzt? Wir müssen miteinander reden!,Impfpflicht jetzt? Wir müssen miteinander reden!,Impfpflicht oder nicht: Wir müssen miteinander...,Der Ethikrat befürwortet mehrheitlich eine all...,Als am 1. Januar 2019 Heribert Prantl in seine...,de,www.faz.net,aktuell_feuilleton_debatten_impfpflicht-jetzt-...,https://www.faz.net/aktuell/feuilleton/debatte...,https://media1.faz.net/ppmedia/aktuell/wissen/...,...,Institut Für Demoskopie Allensbach,Oliver Georgi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,"Weltraum-Teleskop ""James Webb"": Astronaut Walt...","Weltraum-Teleskop ""James Webb"": Astronaut Walt...","""Da kann noch viel schiefgehen""","Das ""James Webb""-Teleskop ist erfolgreich ins ...","Das ""James Webb""-Teleskop ist erfolgreich ins ...",de,www.welt.de,wissenschaft_video235869636_Weltraum-Teleskop-...,https://www.welt.de/wissenschaft/video23586963...,https://img.welt.de/img/wissenschaft/mobile235...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1468,Studie aus Hamburg: Organschäden auch bei mild...,Studie aus Hamburg: Organschäden auch bei mild...,Organschäden auch bei mildem Corona-Verlauf,Eine Studie des Universitätsklinikums Hamburg-...,Eine Studie des Universitätsklinikums Hamburg-...,de,www.welt.de,wissenschaft_video236043420_Studie-aus-Hamburg...,https://www.welt.de/wissenschaft/video23604342...,https://img.welt.de/img/regionales/hamburg/mob...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1469,"""James Webb"" startet seine Mission",»James Webb« startet seine Mission - DER SPIEGEL,»James Webb« startet seine Mission,Von Französisch-Guayana ins All: Das Weltraumt...,NaN,de,www.spiegel.de,wissenschaft_weltall_james-webb-startet-seine-...,https:

In [10]:
df.to_csv(path_to_data / "df.csv", index=False)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1257 entries, 0 to 1471
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          1257 non-null   object        
 1   title_page     1257 non-null   object        
 2   title_rss      1257 non-null   object        
 3   description    1257 non-null   object        
 4   text_main      932 non-null    object        
 5   language       1257 non-null   category      
 6   source_domain  1257 non-null   category      
 7   filename       1257 non-null   object        
 8   url            1257 non-null   object        
 9   image_url      1257 non-null   object        
 10  category_0     1257 non-null   object        
 11  category_1     914 non-null    object        
 12  category_2     412 non-null    object        
 13  category_3     71 non-null     object        
 14  category_4     13 non-null     object        
 15  date_download  1257 n

In [12]:
# generate DB with only wirtschaft categories
df_entities = pd.read_csv(path_to_data / "df_entities.csv")
query = "(category_0=='wirtschaft' or category_1=='wirtschaft' or category_2=='wirtschaft' or category_3=='wirtschaft' or category_4=='wirtschaft') and text_main.notna()"
df_wirtschaft = df_entities.query(query)
df_wirtschaft.to_csv(path_to_data / "df_wirtschaft.csv", index=False)